<a href="https://colab.research.google.com/github/Nick088Official/versatile_audio_super_resolution/blob/main/Audio_SR_WEB_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio SR WEB UI

Versatile Audio Super Resolution (any -> 48kHz)

*NOTE:* Be sure to be using a GPU, it will crash on CPU because of the free RAM being too less

In [ ]:
#@title Install
#@markdown If you get a warning that you have to restart the runtime, you don't have to, just click cancel
from IPython.display import clear_output
!git clone https://github.com/Nick088Official/versatile_audio_super_resolution

%cd versatile_audio_super_resolution

!pip install -r requirements.txt
clear_output()
print("Installed!")

In [ ]:
#@title Run UI
import re


#@markdown The type of tunnel you wanna use for seeing the public link, so if the API of one of them is down, you can use the other one.
Tunnel = "Gradio" #@param ["Gradio", "Ngrok", "Cloudfare", "LocalTunnel"]

#@markdown Also when using Ngrok, Cloudfare or LocalTunnel as the Tunnel, you need to wait for the Local URL to appear, and only after that click on the  Public URL above it.

#@markdown Use the option under this only if you chose Ngrok as the Tunnel:

#@markdown You can get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new.

ngrok_tunnel_authtoken = "" #@param {type:"string"}


file_path_ui = "/content/versatile_audio_super_resolution/app.py"


if Tunnel == "Gradio":
  !sed -i 's/share=False/share=True/g' $file_path_ui
elif Tunnel == "Ngrok":
  !sed -i 's/share=True/share=False/g' $file_path_ui
  !pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_tunnel_authtoken)
  http_tunnel = ngrok.connect(7860, bind_tls=True)
  clear_output()
  print("Ngrok Tunnel Public URL:", http_tunnel.public_url)
elif Tunnel == "Cloudfare":
  !sed -i 's/share=True/share=False/g' $file_path_ui
  # download cloudfare
  !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  !rm -rf nohup.out
  import time
  # Run cloudflare
  !nohup cloudflared tunnel --url localhost:7860 &
  clear_output()
  time.sleep(5)
  # Find and print the Cloudflare URL with a prefix
  cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
  print(f"Cloudfare Tunnel Public URL: {cloudflare_url[0]}")
elif Tunnel == "LocalTunnel":
  !sed -i 's/share=True/share=False/g' $file_path_ui
  # install
  !npm install -g localtunnel
  import time
  import urllib
  # run localtunnel
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 7860 >> url.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  clear_output()
  print(f"LocalTunnel Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  print(f'LocalTunnel Password: {endpoint_ip}')

!python app.py